<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Final Challenge - A

## Implement a simple estimator


A core design element in `river` is that it should be easy to create new methods or extend existing methods.

In the following example, we show how to implement the `MajorityClassClassifier`.

The Majority Class is one of the simplest classifiers: it predicts the class of a new sample to be the most frequent at that point in the stream.

It is used mostly as a baseline, but also as a default classifier at the leaves of decision trees.

`River` provides a set of `mixins` for different learning tasks. In this case, we want to create a classifier so we extend the `base.Classifier` mixin as follows: 

In [ ]:
from river import base
from collections import Counter

class MajorityClassClassifier(base.Classifier):
    def __init__(self):
        # Initialization
        self._counts = Counter()
    
    def learn_one(self, x:dict, y):
        # Learn one sample
        self._counts.update([y])

    def predict_one(self, x:dict):
        # Predict class for one sample
        mc = self._counts.most_common()
        if mc:
            return mc[0][0]
        return 0   # Counter is empty

    def predict_proba_one(self, x:dict):
        # Predict class probability for one sample
        total = sum(self._counts)
        y_proba = {}
        if total > 0:    # Protect division by zero
            for x, cnt in self._counts.items():
                y_proba[x] = cnt / total
        return y_proba
    
    @property
    def _multiclass(self):
        return True

## SEA Stream
---
Each observation in [SEA](https://riverml.xyz/latest/api/synth/SEA/) generator is composed of 3 features. Only the first two features are relevant. The target is binary, and is positive if the sum of the features exceeds a certain threshold.

Synthetic data generators are useful since they do not store the data but generate it on demand. Although data generators are infinite, for this example, we limit the number of samples generated using the `take()` method.

In [ ]:
from river import synth

stream = synth.SEA(seed=42).take(20000)

In [ ]:
from river.metrics import Accuracy
from river.evaluate import progressive_val_score

model = MajorityClassClassifier()
metric = Accuracy()

progressive_val_score(dataset=stream, model=model, metric=metric, print_every=1000)

Which is consistent with the class distribution as the data is slightly unbalanced in favor of the `True` class.

In [ ]:
model._counts[True] / sum(model._counts.values())

As mentioned, the performance of simple methods such as the majority class classifier is as a baseline.

For example, we can compare it against the Gaussian Naive Bayes classifier:

In [ ]:
from river.naive_bayes import GaussianNB

stream = synth.SEA(seed=42).take(20000)

model = GaussianNB()
metric = Accuracy()

progressive_val_score(dataset=stream, model=model, metric=metric, print_every=1000)

## No Change Classifier

---
Implement the `NoChangeClassifier`, which predicts the label for a new instance to be the true label of the previous instance. Like the Majority Class classifier, it does not require the instance features, so it is very easy to implement. In the intrusion detection case where long passages of “no intrusion” are followed with briefer periods of “intrusion,” this classifier makes mistakes only on the boundary cases, adjusting quickly to the consistent pattern of labels.

In [ ]:
from river import base

class NoChangeClassifier(base.Classifier):
    def __init__(self):
        # Initialization
        self._last_y = 0
        self._classes = []
    
    def learn_one(self, x:dict, y):
        # Learn one sample
        <FILL-IN>

    def predict_one(self, x:dict):
        # Predict class for one sample        
        <FILL-IN>

    def predict_proba_one(self, x:dict):
        # Predict class probability for one sample        
        y_proba = {}        
        <FILL-IN>
        return y_proba
    
    @property
    def _multiclass(self):
        return True

In [ ]:
model = NoChangeClassifier()
metric = Accuracy()
stream = synth.SEA(seed=42).take(20000)

progressive_val_score(dataset=stream, model=model, metric=metric, print_every=1000)